# Equilibrate Test Cases
Notebook to develop and test Python code that substitutes for the Objective-C Equilibrate Class

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys

In [ ]:
from thermoengine import core, phases, model, equilibrate

## Create phase for equilibrium assemblages

In [ ]:
modelDB = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')

The Berman model database provides the SWIM water model by default.  Instead, override that choice by instantiating the MELTS 1.0.2 water model directly.

In [ ]:
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

## Define elements in system and phases in system

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
phs_sys = [Liquid, Feldspar, Water, Quartz]
#phs_sys = [Liquid, Water, Quartz]
#phs_sys = [Liquid, Feldspar, Water]

## Composition of the system

In [ ]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5
}

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')

In [ ]:
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)
blk_cmp

# Simple Gibbs energy minimization

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
t = 1050.0
p = 1750.0
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
state = equil.execute(t-10.0, p, state=state, debug=0)
state.print_state()

In [ ]:
state = equil.execute(t-20.0, p, state=state, debug=0)
state.print_state()

In [ ]:
state = equil.execute(t-25.0, p, state=state, debug=0)
state.print_state()

# Korzhinskii potential minimization (O<sub>2</sub> constrained)

In [ ]:
def mu0O2(t, p):
    tr = 298.15
    hs = 23.10248*(t-tr) + 2.0*804.8876*(np.sqrt(t)-np.sqrt(tr)) - 1762835.0*(1.0/t-1.0/tr) \
       - 18172.91960*np.log(t/tr) + 0.5*0.002676*(t*t-tr*tr)
    ss = 205.15 + 23.10248*np.log(t/tr)  - 2.0*804.8876*(1.0/np.sqrt(t)-1.0/np.sqrt(tr)) \
       - 0.5*1762835.0*(1.0/(t*t)-1.0/(tr*tr)) + 18172.91960*(1.0/t-1.0/tr) + 0.002676*(t-tr)
    return hs - t*ss
def muNNO(t, p):
    logNNO = -25018.7/t + 12.981 + 0.046*(p-1.0)/t - 0.5117*np.log(t)
    return 8.3144598*t*np.log(10.0)*logNNO
def muO2(t, p, state):
    return mu0O2(t, p) + muNNO(t, p)

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys, lagrange_l=[({'O':2.0},muO2)])

In [ ]:
np.set_printoptions(linewidth=200, precision=1)

In [ ]:
t = 1050.0
p = 1750.0
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
state = equil.execute(t-10.0, p, state=state, debug=0)
state.print_state()

In [ ]:
state = equil.execute(t-20.0, p, state=state, debug=0)
state.print_state()

# Korzhinskii potential minimization (H<sub>2</sub>O constrained)

In [ ]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5 # water saturated
}
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)

In [ ]:
def muH2O(t, p, state):
    return Water.gibbs_energy(t, p)

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys, lagrange_l=[({'H':2.0,'O':1.0},muH2O)])
np.set_printoptions(linewidth=200, precision=1)

In [ ]:
t = 1050.0
p = 1750.0
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

# Simple Enthalpy minimization (Entropy is constrained)

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
t = 1050.0
p = 1750.0
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
delta_dGdT = 0.0
dGdT = state.dGdT(t,p)
def con(t, p, state):
    return dGdT + delta_dGdT
print (con(t, p, None), state.dGdT(t,p))

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys, lagrange_l=[('T',con)])
np.set_printoptions(linewidth=200, precision=1)

In [ ]:
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
delta_dGdT = 5.0
state = equil.execute(t, p, state=state, debug=0)
state.print_state()

# Simple Helmholtz energy minimization (Volume is constrained)

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
t = 1050.0
p = 1750.0
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
delta_dGdP = 0.0
dGdP = state.dGdP(t,p)
def con(t, p, state):
    return dGdP + delta_dGdP
print (con(t, p, None), state.dGdP(t,p))

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys, lagrange_l=[('P',con)])
np.set_printoptions(linewidth=200, precision=1)

In [ ]:
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
delta_dGdP = 0.1
state = equil.execute(t, p, state=state, debug=0)
state.print_state()

In [ ]:
delta_dGdP = 0.2
state = equil.execute(t, p, state=state, debug=0)
state.print_state()

In [ ]:
delta_dGdP = 0.3
state = equil.execute(t, p, state=state, debug=0)
state.print_state()

In [ ]:
delta_dGdP = 0.4
state = equil.execute(t, p, state=state, debug=0)
state.print_state()

# Internal energy minimization (Entropy and volume are constrained)

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
t = 1050.0
p = 1750.0
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
delta_dGdT = 0.0
dGdT = state.dGdT(t,p)
def conT(t, p, state):
    return dGdT + delta_dGdT
print (conT(t, p, None), state.dGdT(t,p))
delta_dGdP = 0.0
dGdP = state.dGdP(t,p)
def conP(t, p, state):
    return dGdP + delta_dGdP
print (conP(t, p, None), state.dGdP(t,p))

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys, lagrange_l=[('T',conT),('P',conP)])
np.set_printoptions(linewidth=200, precision=1)

In [ ]:
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
delta_dGdT = 5.0
delta_dGdP = 0.1
state = equil.execute(t, p, state=state, debug=0)
state.print_state()